<a href="https://colab.research.google.com/github/Gakonnn/Hackathon/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [15]:
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering
from datasets import load_dataset, Dataset
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd



tokenizer = RobertaTokenizerFast.from_pretrained('nur-dev/roberta-kaz-large')
model = RobertaForQuestionAnswering.from_pretrained('nur-dev/roberta-kaz-large')

ds = load_dataset("Kyrmasch/sKQuAD")

train_data = ds['train']

train_df = pd.DataFrame(train_data)

train_df, val_df = train_test_split(train_df, test_size=0.2)

train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)

def preprocess_data(examples):
    questions = examples["question"]
    contexts = examples["context"]
    answers = examples["answer"]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=256,
        truncation=True,
        padding="max_length"
    )

    start_positions = []
    end_positions = []

    for context, answer in zip(contexts, answers):
        if isinstance(answer, list) and len(answer) > 0:
            first_answer = answer[0]
            start = context.find(first_answer)
            if start != -1:
                start_positions.append(start)
                end_positions.append(start + len(first_answer))
            else:
                start_positions.append(-1)
                end_positions.append(-1)
        elif isinstance(answer, str):
            start = context.find(answer)
            if start != -1:
                start_positions.append(start)
                end_positions.append(start + len(answer))
            else:
                start_positions.append(-1)
                end_positions.append(-1)
        else:
            start_positions.append(-1)
            end_positions.append(-1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs



tokenized_train_dataset = train_data.map(preprocess_data, batched=True)
tokenized_val_dataset = val_data.map(preprocess_data, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
)

trainer.train()

eval_results = trainer.evaluate(tokenized_val_dataset)
print("Baseline F1 and EM:", eval_results)


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at nur-dev/roberta-kaz-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,No log,3.621649
2,No log,3.668553
3,3.759000,3.742202


Baseline F1 and EM: {'eval_loss': 3.742201566696167, 'eval_runtime': 3.2082, 'eval_samples_per_second': 62.34, 'eval_steps_per_second': 15.585, 'epoch': 3.0}


In [16]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r ./results /content/drive/MyDrive/roberta-kaz-large-finetuned



Mounted at /content/drive


In [4]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer', 'topic'],
        num_rows: 1000
    })
})
